In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import chromadb
from chromadb.utils import embedding_functions
from transformers import pipeline, PretrainedConfig,AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, Markdown
import torch
from huggingface_hub import login
# from kaggle_secrets import UserSecretsClient
import os

c:\Users\I520292\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\I520292\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\I520292\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\I520292\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_

In [3]:
# loading data
datafile = pd.read_csv("kaggle_winning_solutions_methods.csv")
datafile.info()

datafile.head()

# Data cleaning
datafile = datafile[['place','competition_name','metric','year','writeup','methods']] # omit not required columns
datafile.drop_duplicates(inplace=True) # remove duplicate rows
datafile.info() 


datafile['writeup'] = datafile['writeup'].apply(lambda x: BeautifulSoup(x,'html.parser').get_text())
datafile.reset_index()
datafile.head()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13164 entries, 0 to 13163
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   link              13164 non-null  object
 1   place             13164 non-null  int64 
 2   competition_name  13164 non-null  object
 3   prize             13164 non-null  object
 4   team              13164 non-null  object
 5   kind              13164 non-null  object
 6   metric            12639 non-null  object
 7   year              13164 non-null  int64 
 8   nm                13164 non-null  int64 
 9   writeup           13164 non-null  object
 10  num_tokens        13164 non-null  int64 
 11  methods           13164 non-null  object
 12  cleaned_methods   13164 non-null  object
dtypes: int64(4), object(9)
memory usage: 1.3+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1262 entries, 0 to 13154
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype

,place,competition_name,metric,year,writeup,methods
0,2,Google - Isolated Sign Language Recognition,PostProcessorKernelDesc,2023,TLDR\nWe used an approach similar to audio spe...,"['EfficientNet-B0', 'Data Augmentation', 'Norm..."
18,3,Google - Isolated Sign Language Recognition,PostProcessorKernelDesc,2023,We used an ensemble of six conv1d models and t...,"['Conv1D', 'Transformers', 'Data preprocessing..."
23,4,Google - Isolated Sign Language Recognition,PostProcessorKernelDesc,2023,I would like to thank the organizers and all t...,"['XY coordinates', 'Normalization', 'Flip', '1..."
40,5,Google - Isolated Sign Language Recognition,PostProcessorKernelDesc,2023,Here is a quick overview of the 5th-place solu...,"['Augmentation', 'Transformer model', 'Preproc..."
45,6,Google - Isolated Sign Language Recognition,PostProcessorKernelDesc,2023,"Thanks to both, the organizers of this competi...","['MLP', 'Encoder', 'Transformer', 'Convolution..."


In [4]:

# File structure for indexing in vector database
total_entries = len(datafile) # total number of data points

# Document for writeup
documents = [f"{datafile.iloc[i]['place']} place solution :  \n competition name : {datafile.iloc[i]['competition_name']} \n metric : {datafile.iloc[i]['metric']} \n year : {datafile.iloc[i]['year']} \n writeup : {datafile.iloc[i]['writeup']} \n methods : {datafile.iloc[i]['methods']}" for i in range(total_entries)]

# Metadata to save for each document
metadatas =  [
            {
            'place':str(datafile['place'].iloc[j]),
            'competition_name':str(datafile['competition_name'].iloc[j]),
            'year':str(datafile['year'].iloc[j])
            } for j in range(total_entries)
            ]


# Configure vector database
client = chromadb.Client() # initiate client
ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name = 'all-MiniLM-L6-v2') # Embedding function for documents

c:\Users\I520292\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [5]:
# Create a vector database
collection = client.create_collection(
    name = 'kaggle_competition_solutions',
    embedding_function = ef ,
    metadata={"hnsw:space": "l2"}
)


# Ingest documnets into vector database
collection.add(
    documents = documents,
    ids = [str(i) for i in range(total_entries)],
    metadatas = metadatas
)



# Setup the environment
access_token_read = os.getenv('HUGGINGFACE_TOKEN')
login(token = access_token_read)

# Load the model
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it",device_map="cuda")



ImportError: The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywidgets` module: `pip install ipywidgets`.